In [1]:
import time, array, random, copy, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

import PreEmo

Assuming given user Input
Duration = 5 weeks
Budget = 4000
Unwanted Regions = [Afghanistan, Pacific]
Preference activities = [hiking, beach, watersports, culinary, entertainment]
Preferred Travel Months = [apr, may, jun]

### Now pre-initialization

First we use the similarity metric to get a ranking of regions
$$
\begin{align*}
sim_{feature} (f_q, f_c) = 1 - \frac{\lvert f_q - f_c \rvert} {max(f_q , f_c)}
\end{align*}
$$

$$
\begin{align*}

similarity (q, c) = \frac{\sum_{i=1}^{n} w_i * sim_i(q_i, c_i)} {\sum_{i=1}^{n} w_i}
\end{align*}
$$

similarity current region and given input
all possible preferences is n
f_q = 1
f_c = actual score for preference i



We do a Max Min Normalization of the scores 

$$
\begin{align*}
X_{ij} = \frac{X_{ij} - X_j^{min}}{X_j^{max}-X_j^{min}}
\end{align*}
$$

In [2]:
start_df = PreEmo.get_start_df()

In [3]:
by_region = start_df.groupby(['Region', 'category', 'month_name'])
'''
[
    safety from crime,
    hiking,
    shopping,
    jan,
    mar
]
'''

'\n[\n    safety from crime,\n    hiking,\n    shopping,\n    jan,\n    mar\n]\n'

To do: transform new dataset into compatible array matrix as needed 

Implementing the Moop Problem

$$
\begin{align*}maximize \qquad  &z_k(x) = \sum_{i=1}^n x_i\ s_{ik}, \hspace{1cm} k = 1,...,p \\
    subject \ to \qquad &\sum_{i=1}^n x_i d_i  \leq D \\
    &\sum_{i=1}^n x_i\ b_i \leq B \\
    &\sum_{j \in N} x_{ij} \geq 1 \\ 
    &u_i(x_i) \geq U \\
    x_i \in \{0,1\}, \qquad &\forall \ 1 \leq i \leq n
\end{align*}
$$